In [1]:
# Event Synchronization Analysis following 
# https://github.com/pik-copan/pyunicorn/blob/master/tests/test_funcnet/TestEventSyncronization.py
# and 
# https://vis.caltech.edu/~rodri/papers/event_synchro.pdf


In [2]:
import sys, string
from matplotlib import rc
import numpy as np
import pylab as pl
import netCDF4
import time as t
import datetime
from dateutil.parser import parse
from pylab import load, meshgrid, title, arange, show
from netcdftime import utime
import scipy.io
import matplotlib as mpl
import argparse
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
import datetime as dt
from netCDF4 import num2date, date2num
import random
from mpi4py import MPI
#import Nio
#from pyhdf.SD import SD


In [3]:
def EventSync(es1, es2, taumax):
    """
    Compute non-vectorized event synchronization
    :type es1: 1D Numpy array
    :arg es1: Event series containing '0's and '1's
    :type es2: 1D Numpy array
    :arg es2: Event series containing '0's and '1's
    :float return: Event synchronization es2 to es1
    """
    ex = np.arange(len(es1))[es1 == 1]
    ey = np.arange(len(es2))[es2 == 1]
    lx = len(ex)
    ly = len(ey)

    count = 0
    if lx!=0 and ly!=0:
        for m in range(1, lx-1):
            for n in range(1, ly-1):
                dst = ex[m] - ey[n]

                if abs(dst) > taumax:
                    continue
                elif dst == 0:
                    count += 0.5
                    continue

              # finding the dynamical delay tau
                tmp = ex[m+1] - ex[m]
                if tmp > ex[m] - ex[m-1]:
                    tmp = ex[m] - ex[m-1]
                tau = ey[n+1] - ey[n]
                if tau > ey[n] - ey[n-1]:
                    tau = ey[n] - ey[n-1]
                if tau > tmp:
                    tau = tmp
                tau = tau / 2

                if dst > 0 and dst <= tau:
                    count += 1

    #print("count = ",count) 
    #print("Q = ",np.sqrt((lx-2) * (ly-2))) 
    #print("lx,ly,Q =",lx,ly,count) 
    if lx!=0 and ly!=0:
        return count / np.sqrt((lx) * (ly))
      #return count / np.sqrt((lx-2) * (ly-2))
    else:
        return 0.0


In [4]:
plv_pi = np.genfromtxt ('plv_pi.csv', delimiter=",")
plv_hist = np.genfromtxt ('plv_hist.csv', delimiter=",")[:11964,]
plv_99p_pi = np.genfromtxt('plv_95p_pmip3_ipsl_pi.csv', delimiter=",")
plv_99p_hist = np.genfromtxt('plv_95p_pmip3_ipsl_hist.csv', delimiter=",")[:11964,]
volc_sigl = -1*np.genfromtxt ('sigl.txt', delimiter=",")

volc_data = volc_sigl[1:998]
volc_data_mon = np.zeros((997*12))
volc_data_mon[0:6] = volc_data[0]
volc_data_mon[11958:11964] = volc_data[-1]
for yyyy in range(996):
    #print(yyyy)
    volc_data_mon[6+yyyy*12:18+yyyy*12] = volc_data[1+yyyy]

print(plv_99p_pi.shape)
print(plv_99p_hist.shape)
print(volc_data_mon.shape)


(11964,)
(11964,)
(11964,)


In [5]:
es_pi = np.zeros((plv_pi.shape[0]))
es_hist = np.zeros((plv_hist.shape[0]))
es_volc =np.zeros((volc_data_mon.shape[0]))

es_pi[plv_pi>=plv_99p_pi] = 1.0
es_hist[plv_hist>=plv_99p_hist] = 1.0
es_volc[volc_data_mon>=0.1] = 1.0


In [6]:
print(np.sum(es_pi))
print(np.sum(es_hist))
print(np.sum(es_volc))
print(es_volc.shape, es_hist.shape)

936.0
1090.0
1392.0
(11964,) (11964,)


In [9]:
taumax = 60
Q_hist_hv = EventSync(es_hist, es_volc, taumax)
Q_hist_vh = EventSync(es_volc, es_hist,taumax)
Q_hist = Q_hist_hv + Q_hist_vh

In [10]:
Q_hist

0.11203306557596002

In [11]:
def my_shuffle(array):
    random.shuffle(array)
    return array


In [12]:
N=1000
Q_hist_mc = np.zeros(N)
for i in range(N):
    es_volc_mc = my_shuffle(es_volc)
    es_hist_mc = my_shuffle(es_hist)
    Q_hist_hv = EventSync(es_hist_mc, es_volc_mc, taumax)
    Q_hist_vh = EventSync(es_volc, es_hist_mc,taumax)
    Q_hist_mc[i] = Q_hist_hv + Q_hist_vh
    #print(i)

In [13]:
p95 = np.percentile(Q_hist_mc, 95)

In [14]:
p95

0.34990037147274466

In [15]:
# Now taking only those events which occured during warm pdo
es_hist.shape

(11964,)

In [16]:
!pwd

/iitm2/cccr-res/msingh/pmip_data_scripts/agu_volcano/event_synchronization_analysis/bedartha_1


In [17]:
warm_pdo = np.genfromtxt ('pdo_ipsl_pmip3_850_1849_first_diff_es_warm.csv', delimiter=",")[18:-18]
cold_pdo = np.genfromtxt ('pdo_ipsl_pmip3_850_1849_first_diff_es_cold.csv', delimiter=",")[18:-18]

In [18]:
es_hist_warm_pdo = es_hist*warm_pdo
es_hist_cold_pdo = es_hist*cold_pdo

In [19]:
taumax = 60
Q_hist_hv_warm_pdo = EventSync(es_hist_warm_pdo, es_volc, taumax)
Q_hist_vh_warm_pdo = EventSync(es_volc, es_hist_warm_pdo,taumax)
Q_hist_warm_pdo = Q_hist_hv_warm_pdo + Q_hist_vh_warm_pdo

Q_hist_hv_cold_pdo = EventSync(es_hist_cold_pdo, es_volc, taumax)
Q_hist_vh_cold_pdo = EventSync(es_volc, es_hist_cold_pdo,taumax)
Q_hist_cold_pdo = Q_hist_hv_cold_pdo + Q_hist_vh_cold_pdo

In [20]:
print(Q_hist_warm_pdo, Q_hist_cold_pdo)

0.23629716473839135 0.24145364534971214


In [21]:
N=1000
Q_hist_mc_warm_pdo = np.zeros(N)
for i in range(N):
    es_volc_mc = my_shuffle(es_volc)
    es_hist_mc_warm_pdo = my_shuffle(es_hist_warm_pdo)
    Q_hist_hv_warm_pdo = EventSync(es_hist_mc_warm_pdo, es_volc_mc, taumax)
    Q_hist_vh_warm_pdo = EventSync(es_volc, es_hist_mc_warm_pdo,taumax)
    Q_hist_mc_warm_pdo[i] = Q_hist_hv_warm_pdo + Q_hist_vh_warm_pdo
    
Q_hist_mc_cold_pdo = np.zeros(N)
for i in range(N):
    es_volc_mc = my_shuffle(es_volc)
    es_hist_mc_cold_pdo = my_shuffle(es_hist_cold_pdo)
    Q_hist_hv_cold_pdo = EventSync(es_hist_mc_cold_pdo, es_volc_mc, taumax)
    Q_hist_vh_cold_pdo = EventSync(es_volc, es_hist_mc_cold_pdo,taumax)
    Q_hist_mc_cold_pdo[i] = Q_hist_hv_cold_pdo + Q_hist_vh_cold_pdo


In [34]:
p95 = np.percentile(Q_hist_mc_warm_pdo, 95)

In [35]:
p95

0.3086561051367504

In [36]:
p95 = np.percentile(Q_hist_mc_cold_pdo, 95)

In [37]:
p95

0.3021086673699297

In [38]:
# Doing the complete analysis for the volcanically active period 1550-1750 
# The index 1:11964 is from 07/851, so 01/852 represents the index 7
# 01/952 will be represented by 1207, 01/1052 by 2407, 01/1152 by 3607
# 01/1252 by 4807 01/1352 by 6007, 01/1452 by 7207, 01/1552 by 8407
# 01/1652 by 9607, 01/1752 by 10807

# Hence 01/1550 will be represented by 8383 and 01/1750 by 10783
# But indexing in python is from 0 hence 8382:10782

In [41]:
taumax = 60
Q_hist_hv = EventSync(es_hist, es_volc, taumax)
Q_hist_vh = EventSync(es_volc, es_hist,taumax)
Q_hist = Q_hist_hv + Q_hist_vh

(2400,)